In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import warnings
import tika
from tika import parser
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import spacy
import re
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')



In [2]:
base = ['data/AS_', 'data/Ml_', 'data/Bd_', 'data/Hd_']
data = []
num_data = []
for elem in range(0, len(base)):
    count = 0
    if elem != 0:
        final = 10
    else:
        final = 34
        
    for i in range(1, final):
        terminal = base[elem] + str(i) + '.pdf'
        data.append(terminal)
        count+=1
    
    num_data.append(count)



In [3]:
data

['data/AS_1.pdf',
 'data/AS_2.pdf',
 'data/AS_3.pdf',
 'data/AS_4.pdf',
 'data/AS_5.pdf',
 'data/AS_6.pdf',
 'data/AS_7.pdf',
 'data/AS_8.pdf',
 'data/AS_9.pdf',
 'data/AS_10.pdf',
 'data/AS_11.pdf',
 'data/AS_12.pdf',
 'data/AS_13.pdf',
 'data/AS_14.pdf',
 'data/AS_15.pdf',
 'data/AS_16.pdf',
 'data/AS_17.pdf',
 'data/AS_18.pdf',
 'data/AS_19.pdf',
 'data/AS_20.pdf',
 'data/AS_21.pdf',
 'data/AS_22.pdf',
 'data/AS_23.pdf',
 'data/AS_24.pdf',
 'data/AS_25.pdf',
 'data/AS_26.pdf',
 'data/AS_27.pdf',
 'data/AS_28.pdf',
 'data/AS_29.pdf',
 'data/AS_30.pdf',
 'data/AS_31.pdf',
 'data/AS_32.pdf',
 'data/AS_33.pdf',
 'data/Ml_1.pdf',
 'data/Ml_2.pdf',
 'data/Ml_3.pdf',
 'data/Ml_4.pdf',
 'data/Ml_5.pdf',
 'data/Ml_6.pdf',
 'data/Ml_7.pdf',
 'data/Ml_8.pdf',
 'data/Ml_9.pdf',
 'data/Bd_1.pdf',
 'data/Bd_2.pdf',
 'data/Bd_3.pdf',
 'data/Bd_4.pdf',
 'data/Bd_5.pdf',
 'data/Bd_6.pdf',
 'data/Bd_7.pdf',
 'data/Bd_8.pdf',
 'data/Bd_9.pdf',
 'data/Hd_1.pdf',
 'data/Hd_2.pdf',
 'data/Hd_3.pdf',
 'da

In [4]:
# data = ['data/Ml_1.pdf', 'data/Ml_2.pdf', 'data/Ml_3.pdf', 'data/Ml_4.pdf', 'data/Ml_5.pdf', 'data/Ml_6.pdf', 'data/Ml_7.pdf']
biblioteca = []
tika.initVM()
for i in range(0, len(data)):
    
    parsed = parser.from_file(data[i])
    metadados = parsed["metadata"]
    texto = parsed["content"]
    biblioteca.append(texto)

In [5]:
print(len(biblioteca))
print(len(data))

60
60


In [6]:
# f = open ("artigo2.txt", "w")
# f.write (str(texto))
# f.close ()

In [7]:

# f = open ("artigo2.txt", "r")
# texto = f.read()

In [8]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [9]:
def split_pdf(string): 
    li = list(string.split(" ")) 
    return li

In [10]:
def first_clear(data):
    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

    #remove links and mark
    data = [re.sub('(https:\S+)|(@)|(¿)', '', sent) for sent in data]
    
    data = [re.sub('(http:\S+)|(@)|(¿)', '', sent) for sent in data]
    
    data = [re.sub('(www.:\S+)|(@)|(¿)', '', sent) for sent in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # remove spaces in begining and end
    data = [re.sub(' +', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]

    return data

In [11]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [12]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [13]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
def convert(s): 
    a = 0
    # initialization of string to "" 
    new = "" 
  
    # traverse in the string  
    for x in s: 
        new += x + ' '
        a+=1
  
    # return string  
    return new, a

In [15]:

data_cleared = []
train_cleared = []

In [16]:
total_char = 0
for i in range(0, len(biblioteca)):
    texto = biblioteca[i]
    data = split_pdf(texto)
    data = first_clear(data)
    d, a = convert(data)
    
    total_char += a
    data_cleared.append(d)


In [17]:
print(len(data_cleared))
print(len(train_cleared))
print(total_char)

60
0
546022


In [18]:
tagged_data = []
for i, _d in enumerate(data_cleared):
    if i < num_data[0]:
        label = 'As'+ '_' + str(i)
        tagged_data.append(TaggedDocument(words=word_tokenize(_d), tags=[label]))
    elif i < (num_data[0] + num_data[1]):
        label = 'Ml'+ '_' + str(i)
        tagged_data.append(TaggedDocument(words=word_tokenize(_d), tags=[label]))
    elif i < (num_data[0] + num_data[1] + num_data[2]):
        label = 'Bd'+ '_' + str(i)
        tagged_data.append(TaggedDocument(words=word_tokenize(_d), tags=[label]))
    else:
        label = 'Hd'+ '_' + str(i)
        tagged_data.append(TaggedDocument(words=word_tokenize(_d), tags=[label]))


In [19]:
# tagged_data = [TaggedDocument(words=word_tokenize(_d), tags=[str(i)]) for i, _d in enumerate(data_cleared)]

In [20]:
train_data = tagged_data[0:(num_data[0]-7)] + tagged_data[(num_data[0]-1):(num_data[0] + num_data[1]-3)] \
+ tagged_data[(num_data[0] + num_data[1]):(num_data[0] + num_data[1] + num_data[2]-3)] + \
tagged_data[(num_data[0] + num_data[1] + num_data[2]):-3]

In [21]:
test_data = tagged_data[num_data[0]-6:num_data[0]] + tagged_data[(num_data[0] + num_data[1]-2):num_data[0] + num_data[1]] \
+ tagged_data[(num_data[0] + num_data[1] + num_data[2]-2):(num_data[0] + num_data[1] + num_data[2])] +\
tagged_data[-2:]

In [22]:
len(test_data)
for i in range(0, len(test_data)):
    print(test_data[i][1])

['As_27']
['As_28']
['As_29']
['As_30']
['As_31']
['As_32']
['Ml_40']
['Ml_41']
['Bd_49']
['Bd_50']
['Hd_58']
['Hd_59']


In [23]:
max_epochs = 30
vec_size = 300
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(train_data)

for epoch in range(max_epochs):
#     print('iteration {0}'.format(epoch))
    model.train(train_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

Model Saved


In [26]:
model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data

for i in range(0, len(test_data)):
    test = word_tokenize(str(test_data[i][0]))
    v1 = model.infer_vector(test)
    print(test_data[i][1])

    # to find most similar doc using tags
    similar_doc = model.docvecs.most_similar([v1])
    print(similar_doc)


['As_27']
[('As_19', 0.46147218346595764), ('As_21', 0.45939168334007263), ('As_24', 0.45195454359054565), ('Ml_33', 0.4482228755950928), ('Bd_47', 0.44313478469848633), ('As_14', 0.4395579695701599), ('As_18', 0.43617409467697144), ('Bd_42', 0.4332016408443451), ('As_20', 0.42194288969039917), ('As_15', 0.4189050495624542)]
['As_28']
[('As_19', 0.4463167190551758), ('As_15', 0.4249621629714966), ('Ml_35', 0.4203965365886688), ('Bd_46', 0.41479042172431946), ('Ml_34', 0.410614550113678), ('Bd_42', 0.40969881415367126), ('Ml_33', 0.4039025902748108), ('As_21', 0.4037351608276367), ('Bd_47', 0.3969321846961975), ('As_18', 0.39616310596466064)]
['As_29']
[('As_19', 0.4678451418876648), ('As_14', 0.43469321727752686), ('As_18', 0.42145514488220215), ('Ml_34', 0.41988468170166016), ('Ml_33', 0.4135134220123291), ('As_21', 0.4119584560394287), ('Bd_44', 0.41059476137161255), ('Ml_36', 0.40933939814567566), ('Ml_35', 0.4059937596321106), ('Bd_42', 0.4007759392261505)]
['As_30']
[('As_19', 0.5